REF: https://medium.com/@shanlins/spatial-transformer-networks-stn-and-its-implementation-2638d58d41f8

### Why we need STN?

In traditional image processing field, ***rotational invariance or scale invariance*** is of great importance, and actually, there are many feature descriptors such SIFT and SURF famous for their consistent performance against ***affine transformation***. 

> rotational/scale invariance가 일어나지 않는게 좋다는거지?

> affine transformation이란?

The design of CNN could be insensitive to some slight rotation or translation transformation. For example, pooling layer could tolerate the pixel switch inside the pooling window. But the following distorted mnist image may challenge the capability of CNN to extracting the most salient feature in the image.

<img src='./imgs/distoredmnist.png'>

CNN could automatically learn how to code the feature in images, it should be able to learn what affine transform the distorted image should apply to recover its original ‘gesture’. 

### Where we use STN module?

The most primitive usage of STN module is to **restore the distorted shape** like below.

<img src='./imgs/distortedmnist2.png'>

The above example is supervised learning where the labels are the original images, but we could also insert STN module into a neural network such like [this](https://pytorch.org/tutorials/intermediate/spatial_transformer_tutorial.html) . This example is also supervised learning but the labels are the digits instead of original images. That shows that, STN module are somehow able to auto-encode the transformation matrix.

<img src='./imgs/distortedmnist3.png'>


### How to fit a spatial transformer to a network module?


It seems like a trend to ‘layerify’ everything into a nerual network, in other word, put everything into back-propagation procedure, such as CRF-DNN and IOU loss layer.

***Here I would just mentixon the basic idea behind STN and the premise of all algorithm is all the affine transformation happen in 2D space (distinguish with homography transformation in 3D, although I think it could be fitted into CNN as well).***

> homography transformation???

When we consider how to design a CNN playing a role of spatial transformer, from my point of view, the two primary things we should make clear are what are inputs and ouputs. In this problem, input is definitely the original image and output should be the transformed image, so the body of CNN should learn what transformation matrix ought to be applied based on the input. What’s more, this network should include the module which helps map the pixel values from input to output according to the transformation matrix just generated, but this part is not ‘learnable’ but designed by hand.

Formula first.

$$\begin{bmatrix}
x^s\\ y^s\end{bmatrix} =  \begin{bmatrix}
\theta_{11} & \theta_{12} & \theta_{13} \\
\theta_{21} & \theta_{22} & \theta_{23} 
\end{bmatrix} \begin{bmatrix}
x^t \\ y^t \\ I
\end{bmatrix}
$$


***This equation(from input to output) tells us, only using six parameters we define an affine transformation and our goal comes out quite obviously that for each image, our model could output six parameters from one of its layers and these parameters decide how the $[x^t, y^t]$ should be transformed to $[x^s, y^s]$ (e.g rotation, shift, scale). The above work is done by the layer called Localisation Net.***

> ???



<img src='./imgs/st.png'>

More formally, the Localisation net is defined as follows:
* input: feature map U of shape (H, W, C)
* output: transformation matrix θ of shape (6,)
* architecture: fully-connected network or ConvNet as well.

Here we come down with another key point when dealing with this kind of transformation tasks, which matters but tends to be ignorant unless being implemented. For example, what we wanna get a transformed image of 100*100 from an original image of 100*100, mapping function is F. 

Suppose $(x, y)$ with $g$ grayscale in the original image, and $(x’,y’)$ in the transformed image is the mapped point of $(x, y)$. If everything goes well, the pixel value of $(x’, y’)$ should be $g$. But for most of time, we would get a decimal grid indices $(x’, y’)$ which means the pixel value $g$ has no place to settle down in the transformed image.

The general interpolation hardly helps because the premise of using interpolation is already getting the pixel values of anchor points around the decimal points. Plus, generally after an affine transformation, the original image information cannot ‘wrap’ the transformed image totally, which results in some points in transformed image ‘undefined’. 

While ‘back-mapping’ method that maps the points in the destination image back to the original image and applies interpolation method in the scale of that original image, will solve the problem. The implementation asks more for mathematics than programming when handling some fancy tasks such as wrapping a square to a circle.

Therefore, we’d better to clarify the meaning of each term in the first formula. $[x^s, y^s]$ represents one grid index position in input feature map, $[x^t, y^t]$ represents a set of indices covering the whole output feature map and transformation matrix $\theta$ describe how the output is transformed to the input. Plus, [the blog seems to hold different interpretations about that equation from mine.](https://medium.com/@shanlins/spatial-transformer-networks-stn-and-its-implementation-2638d58d41f8)

If localisation net cares only about the spatial relationship between the input and output, the grid generator and sampler are designed to assign values to output feature map pixels. After applying the first formula to all the output feature map grid, we get the fractional grid indices of the input feature map. Rounding the fractional grid indices to some integer indices is not applicable here, because round operation doesn’t support gradient back flowing through. While interpolation operation is compatible with back-propagation.
In sum, grid generator and sampler will realize grid transformation and (bilinear) interpolation in one step. I would go into one very elegant implementation of this module in detail in the next chapter.


### How to implement STN module?

I will only discuss how to implement grid generator and sampler in STN module because localisation net is basically a normal CNN.
Call sub routine in Pytorch